<a href="https://colab.research.google.com/github/martinsadw/brainstorm-helper/blob/main/brainstorm_helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Caso apenas queira executar o modelo:**
- Crie uma Chave de API [(link)](https://aistudio.google.com/app/apikey);
- Adicione a chave à lista de secrets do Google Colab com o nome `GOOGLE_API_KEY`;
- Habilite o acesso da chave pelo notebook;
- Clique em `Ambiente de execução > Executar tudo` ou aperte `Ctrl + F9` e espera até o modelo ser instânciado.

Índice
- [Instruções de uso](#0-instructions-name): Explica como interagir com o modelo
- [Funções auxiliares](#1-auxiliar-name): Define algumas funções extras para facilitar algumas taredas;
- [Criar o modelo](#2-model-name): Cria e instancia o modelo especificando os parametros de configuração;
- [Executar o modelo](#3-execute-name): Executa o modelo e solicita os prompts para interagir com o modelo. É aqui que a diversão começa. 👍

---

<a name="0-instructions-name"></a>
# 0. Instruções de uso

As interações com o modelo funcionam em três etapas: **Elaboração**, **Refinamento** e **Exploração** de ideias.  
É possível digitar "SAIR" (sem aspas, em letras maiúsculas) a qualquer momento para encerrar a interação.

### 1. Elaboração de ideias

Na primeira etapa, tudo o que for enviado para o modelo será adicionado à lista de ideias. É possivel pedir para o modelo remover uma ideia caso tenha sido adicionada incorretamente. Quando terminar de adicionar as ideias diga "Finalizar" para processeguir para a próxima etapa.

### 2. Refinamento de ideias

Após finalizar a primeira etapa, o modelo irá sugerir novas ideias para expandir as ideias sugeridas originalmente. Em seguida o modelo irá resumir a lista em um conjunto reduzido de ideias. Ideias que sejam muito similares serão agregadas em um conjunto de tópicos mais amplos. O modelo seguirá automaticamente para a próxima etapa.

### 3. Exploração de ideias

Por fim, o modelo se disponibilizará a responder perguntas relacionadas as ideias propostas. Pode se pedir mais detalhes sobre algum dos itens, estimativas de custo para a execução das ideias, solicitar que o modelo crie um esboço de implementação das tarefas, entre outras ideias. Sua criatividade é o limite aqui.


<a name="1-auxiliar-name"></a>
# 1. Funções auxiliares

Essas funções são utilizadas ao longo do código e são de um modo geral autoexplicativas pelo nome. Os detalhes podem ser ignorados caso queira apenas testar o modelo.

In [1]:
# Utilizado para imprimir os detalhes do modelo criado de maneira agradável
def print_model(model):
    print('Nome do modelo:', model.model_name, '\n')
    print('Parametros extras')
    print('=================')
    print('Temperatura:', model._generation_config['temperature'])
    print('Top P:', model._generation_config['top_p'])
    print('Top K:', model._generation_config['top_k'])
    print('Número máximo de tokens:', model._generation_config['max_output_tokens'])
    print('')
    print('Instruções do Sistema')
    print('=====================')
    print(str(model._system_instruction.parts[0].text))

In [2]:
import time
import random
import logging
from google.api_core.exceptions import TooManyRequests

# Desabilita messagens de aviso que aparecem quando o erro `TooManyRequests` acontece para não poluir o terminal
logging.getLogger('tornado.access').disabled = True

# Tenta enviar uma mensagem tratando erro de limite de quota
# Implementa a estratégia de Exponential Backoff com Jitter para retentar enviar uma mensagem
def try_send_message(convo, message):
    retries = 0
    while True:
        try:
            convo.send_message(message)
            return
        except TooManyRequests as e:
            if retries >= 5:
                print('Número máximo de tentativas excedido.')
                raise  # Dispara o erro novamente depois de 5 tentativas
            print(f'Erro: Muitas requisições\nTentando novamente {retries+1} / 5...')
            wait_time = random.uniform(2 ** (retries + 1), 2 ** (retries + 2))
            time.sleep(wait_time)
            retries += 1

<a name="2-model-name"></a>
# 2. Criar o modelo

In [3]:
# Baixa e atualiza a biblioteca necessária
!pip install -q -U google-generativeai

In [4]:
# Importa o Python SDK
import google.generativeai as genai
# Usado para obter a chave armazenada no Colab
from google.colab import userdata

# Obtem a chave armazenado com o nome 'GOOGLE_API_KEY' no Colab
# Lembrar de clicar no botão de switch para liberar o acesso para o notebook
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
# Define parametros do modelo
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

In [6]:
# Define as configurações de segurança
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

In [11]:
# Define as instruções de sistema do modelo
# system_instruction_old = 'Você irá auxiliar no processo de brainstorming de ideias.\nEsse processo se dará em quatro etapas.\nCaso o usuário peça, em qualquer etapa, um comando que não esteja descrito além dos especificadas para a respectiva etapa responda com \"Comando inválido\".\n\nEtapa 1: Lista de ideias\n- Você irá receber frases com ideias a serem implementadas.\n- A cada frase recebida, adicione essa frase em uma lista com todas as frases recebidas e imprima essa lista. Apenas adicione o cabeçalho \"Lista de ideias:\" e nada mais.\n- Essa lista poderá ter quantos itens forem necessários, não interrompa a criação da lista.\n- Quando o usuário disser que acabou a sessão de brainstorming, imprima uma ultima vez a lista de ideias e passe para a próxima etapa.\n- Caso o usuário peça para remover alguma ideia especifica, remova ela da lista e imprima novamente a lista.\n\nEtapa 2: Sugestões Extras\n- Quando a sessão de brainstorming acabar adicione uma nova lista de \"Sugestões Extras\".\n- Essa lista conterá novas ideias relacionadas aos item adicionados anteriormente que serão sugeridos por você.\n- Garanta que essa nova lista não seja muito grande, não tendo mais ideias do que a quantidade de ideias da primeira lista.\n- Após imprimir a lista mova para a próxima etapa.\n\nEtapa 3: Consolidação de Ideias\n- Modifique a lista inicial e as sugestões extras de modo a agregar ideias que sejam muito similares.\n- Essa lista poderá conter mais de uma ideia caso sejam suficientemente distintas.\n- Imprima um lista de \"Ideias Principais\" contendo apenas as ideias que passarem por esse filtro.\n- Após imprimir a lista de ideias principais, mova para a próxima etapa.\n\nEtapa 4: Auxilio para a execução das ideias\n- Comece dizendo \"Estou a disposição para dar sugestões de como executar essas ideias\".\n- Responda a novas solicitações relacionadas as ideias propostas, dando dicas de como executar as ideias, adicionando mais detalhes as ideias quando necessário ou respondendo perguntas relacionadas as ideias.\n- Mantenha a lista de \"Ideias Principais\" como o foco do restante da conversa.\n- Não é mais necessário responder com \"Comando inválido\".'
system_instruction = "Você irá auxiliar no processo de brainstorming de ideias.\nEsse processo se dará em quatro etapas.\n\nEtapa 1: Lista de ideias\n- Você irá receber frases com ideias a serem implementadas.\n- A cada frase recebida, adicione essa frase em uma lista com todas as frases recebidas e imprima essa lista. Apenas adicione o cabeçalho \"Lista de ideias:\" e nada mais.\n- Essa lista poderá ter quantos itens forem necessários, não interrompa a criação da lista.\n- Quando o usuário disser algum termo como \"acabou\", \"encerrar\", \"finalizar\" ou outros termos similares, encerre a sessão de brainstorming, imprima uma ultima vez a lista de ideias e passe para a próxima etapa.\n- Quando o usuário disser algum termo como \"remover\" seguido de alguma instrução apontando para algumas das ideias na lista, remova a ideia da lista e imprima novamente a lista.\n- Qualquer outra frase deve ser interpretado como uma ideia a ser adicionado à lista\n\nEtapa 2: Sugestões Extras\n- Quando a sessão de brainstorming acabar adicione uma nova lista de \"Sugestões Extras\".\n- Essa lista conterá novas ideias relacionadas aos item adicionados anteriormente que serão sugeridos por você.\n- Garanta que essa nova lista não seja muito grande, não tendo mais ideias do que a quantidade de ideias da primeira lista.\n- Após imprimir as Sugestões Extras, mova para a próxima etapa.\n\nEtapa 3: Consolidação de Ideias\n- Modifique a lista inicial e as sugestões extras de modo a agregar ideias que sejam muito similares.\n- Essa lista poderá conter mais de uma ideia caso sejam suficientemente distintas.\n- Imprima um lista de \"Ideias Principais\" contendo apenas as ideias que passarem por esse filtro.\n- Após imprimir a lista de Ideias Principais, mova para a próxima etapa.\n\nEtapa 4: Auxilio para a execução das ideias\n- Comece dizendo \"Estou a disposição para dar sugestões de como executar essas ideias\".\n- Responda a novas solicitações relacionadas as ideias propostas, dando dicas de como executar as ideias, adicionando mais detalhes as ideias quando necessário ou respondendo perguntas relacionadas as ideias.\n- Mantenha a lista de \"Ideias Principais\" como o foco do restante da conversa.\n"

In [12]:
# Instancia o modelo com as configurações especificadas
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

<a name="3-execute-name"></a>
# 3. Executar o modelo

Estes são os detalhes do modelo usado aqui. As Intruções do Sistema podem ser consultadas caso tenha curiosidade de entender quais foram as regras definidas para o modelo.

In [13]:
print_model(model)

Nome do modelo: models/gemini-1.5-pro-latest 

Parametros extras
Temperatura: 1
Top P: 0.95
Top K: 0
Número máximo de tokens: 8192

Instruções do Sistema
Você irá auxiliar no processo de brainstorming de ideias.
Esse processo se dará em quatro etapas.

Etapa 1: Lista de ideias
- Você irá receber frases com ideias a serem implementadas.
- A cada frase recebida, adicione essa frase em uma lista com todas as frases recebidas e imprima essa lista. Apenas adicione o cabeçalho "Lista de ideias:" e nada mais.
- Essa lista poderá ter quantos itens forem necessários, não interrompa a criação da lista.
- Quando o usuário disser algum termo como "acabou", "encerrar", "finalizar" ou outros termos similares, encerre a sessão de brainstorming, imprima uma ultima vez a lista de ideias e passe para a próxima etapa.
- Quando o usuário disser algum termo como "remover" seguido de alguma instrução apontando para algumas das ideias na lista, remova a ideia da lista e imprima novamente a lista.
- Qualquer 

Algumas sugestões de ideias para serem adicionadas a Lista de Ideias caso queira apenas testar o modelo:
- Criar um modelo para auxiliar em um brainstorming
- Criar um site para organizar tarefas
- Permitir adicionar membros e atribuir esse membros às tarefas
- Permitir criar grupos de trabalho
- Criar um modelo para organizar reuniões

Leia as [Instruções de uso](#0-instructions-name) para entender como interagir com o modelo

In [14]:
from IPython.display import clear_output

convo = model.start_chat(history=[])
end_msg = 'SAIR'
prompt_msg = f'Digite "{end_msg}" para encerrar\nPrompt: '

prompt = input(prompt_msg)
while prompt != end_msg:
    try_send_message(convo, prompt)
    clear_output()
    print(convo.last.text)
    prompt = input(prompt_msg)
    print('Processando...\n')

Lista de ideias:
- Criar um modelo para auxiliar em um brainstorming 
- Criar um site para organizar tarefas 
- Permitir adicionar membros e atribuir esse membros às tarefas 
- Permitir criar grupos de trabalho 
- Criar um modelo para organizar reuniões 

Sugestões Extras:
- No modelo para auxiliar em brainstormings, adicione uma ferramenta para votação das ideias
- No site para organizar tarefas, adicione um sistema de notificações para avisar sobre deadlines ou novas tarefas 
- Permita que os usuários criem modelos personalizados para organizar reuniões 
- Adicione uma agenda compartilhada para os membros dos grupos de trabalho 
- Crie um sistema de recompensas para os membros mais produtivos 

Ideias Principais:
- Criar uma plataforma online para auxiliar na organização e produtividade de equipes, com ferramentas para brainstorming, gestão de tarefas, criação de grupos de trabalho e organização de reuniões. 

Estou a disposição para dar sugestões de como executar essas ideias 

Digi